In [1]:
# importing dependencies
import openai
import os
import langchain
import pinecone 
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI

/usr/local/python/3.12.1/lib/python3.12/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
#loading my env apis
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
## Reading the document
def read_pdfs(directory):
    load_file=PyPDFDirectoryLoader(directory)
    documents=load_file.load()
    return documents

files = read_pdfs('document/')
len(files)

6

In [4]:

# Divide the docs into chunks
def chunked_data(files, chunk_size=800, chunk_overlap=50):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunked_docs = text_splitter.split_documents(files)  # Correctly process the documents
    return chunked_docs  # Return the processed chunked documents



# Check the length of chunked docs
chunked_docs = chunked_data(files=files)  
chunked_length = len(chunked_docs)
print(f"Number of chunks: {chunked_length}")


Number of chunks: 794


In [5]:
## Embedding the document
embeddings = OpenAIEmbeddings(api_key=os.environ['OPENAI_API_KEY'])
embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x7e0f6ae09e80>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7e0f54964a10>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

creating the pinecoin index

In [ ]:
from pinecone import Pinecone
from pinecone import ServerlessSpec

pc = Pinecone(api_key="pcsk_3NTMPs_G3RnBjuJTgiQAtM1Xtv2xDuXyVGb5PSVvoLD9cADzntogrqxCu28m5TNdtA21VG")

pc.create_index(
  name="rag",
  dimension=1536,
  metric="cosine",
  spec=ServerlessSpec(
    cloud="aws",
    region="us-east-1"
  ),
  deletion_protection="disabled"
)


pushing the chched data into pinecoin